In [59]:
import pandas as pd
import datetime
import time
from math import sin, cos, sqrt, atan2, radians
import folium
from numpy.ma.core import ceil
from scipy.interpolate import interp1d


def get_color(min_val, max_val, val):
  mapper = interp1d([min_val, max_val], [0,255])
  color_val = mapper(val)
  color_val = ceil(color_val)
  #print(color_val)
  hex_val = hex(int(color_val))
  hex_val = hex_val[2:]
  #print(hex_val)
  return '#0000'+hex_val


def find_distance(lat1, lon1, lat2, lon2):
  # approximate radius of earth in km
  R = 6373.0

  lat1 = radians(lat1)
  lon1 = radians(lon1)
  lat2 = radians(lat2)
  lon2 = radians(lon2)

  dlon = lon2 - lon1
  dlat = lat2 - lat1

  a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  c = 2 * atan2(sqrt(a), sqrt(1 - a))

  distance = R * c
  return distance*1000


def get_unixtime(date):
    date_time = datetime.datetime(int(date[0]), int(date[1]), int(date[2]), int(date[3]), int(date[4]), int(date[5]))
    return time.mktime(date_time.timetuple())


class drifters:

    def __init__(self, file_name, deploy_time, recovery_time, df = 1, map = 1):
        self.file_name = file_name
        self.deploy_time = deploy_time
        self.recovery_time = recovery_time
        with open(self.file_name) as src_file:
            #print(src_file.encoding)
            if src_file.encoding != 'UTF-8':
                with open(self.file_name, 'w+b') as dest_file:
                    src_file.read().decode(src_file.encoding).encode('utf-8')

        self.df = pd.read_csv(file_name, encoding = 'utf-8')
        #print(self.df.head())
        
        '''Adding the unix time stamp to the df''' 
        
        self.df = self.df.dropna()
        self.df = self.df.reset_index(drop=True)

        unix_list = []
        for index, rows in self.df.iterrows():
            this_time = rows['DeviceDateTime']
            this_time = this_time.split(' ')
            date = this_time[0].split('-')
            tme = this_time[1].split(':')
            date = date+tme
            
            unix_list.append(get_unixtime(date))
            
        self.df['unix_timestamp']=unix_list

        self.df = self.df[self.df['unix_timestamp']>self.deploy_time]
        
        self.df = self.df[self.df['unix_timestamp']<self.recovery_time]
        self.df = self.df.sort_values(by=['unix_timestamp'])
        self.df = self.df.reset_index(drop=True)
        #print(self.df.head())


        ''' calculate velocities'''
        velocity_list = []
        
        for index, rows in self.df.iterrows():
            if index==0:
                velocity_list.append(0)
                lat1 = rows['Latitude']
                lon1 = rows['Longitude']
                t1 = rows['unix_timestamp'] 
            
            else:
                lat2 = rows['Latitude']
                lon2 = rows['Longitude']
                t2 = rows['unix_timestamp'] 
                if t2==t1:
                    
                    velocity_list.append(this_vel)
                    continue
                this_distance = find_distance(lat1, lon1, lat2, lon2)
                this_vel = this_distance/(t2-t1)
                velocity_list.append(this_vel)
                lat1 = lat2
                lon1 = lon2
                t1 = t2
        
        self.df['vel(m/s)'] = velocity_list
        #print(self.df.head())

    def plot(self):
        vel = self.df['vel(m/s)'].to_list()
        max_vel = max(vel)
        min_vel = min(vel)

        map = folium.Map(location = [43.125832, 5.930556], tiles='OpenStreetMap' , zoom_start = 12)
        folium.raster_layers.WmsTileLayer(url= 'https://ows.emodnet-bathymetry.eu/wms',
                                  layers = 'mean_rainbowcolour',
                                  opacity = 0.3,
                                  transparent = True, 
                                  control = True,
                                  fmt="image/png",
                                  name = 'bathymetry',
                                  overlay = True,
                                  show = True,
                                  ).add_to(map)
        folium.LayerControl().add_to(map)
        folium.CircleMarker(
            location=[43.125832, 5.930556],
            radius=2).add_to(map)

        for index, rows in self.df.iterrows():
            folium.CircleMarker(location=[rows['Latitude'], rows['Longitude']], color=  get_color(min_vel, max_vel, rows['vel(m/s)']),radius=-1, tooltip = str(rows['vel(m/s)'])+' m/s' ).add_to(map)

        self.map = map



In [60]:

drf = drifters('/content/drifter-10_11_22-9666.csv', deploy_time= 1665476880, recovery_time= 1665490380)
drf.plot()
drf.map

In [ ]:
pd.read_csv()

In [ ]:
!pip install chardet


In [34]:
with open('a.csv') as src_file:
        print(src_file.encoding)

UTF-8
